In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam, SGD, Adagrad, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, Callback
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore', category=DeprecationWarning)

#### Preprocessing

In [3]:
train, test = [], []
z_train, z_test = [], []
chars =  ['a', 'bA', 'dA', 'lA', 'tA']
# chars = ["a", "ai", "bA", "lA", "tA"]

def normalize(seq):
    min_x, max_x = min(el[0] for el in seq), max(el[0] for el in seq)
    min_y, max_y = min(el[1] for el in seq), max(el[1] for el in seq)
    for el in seq:
        el[0] = (el[0] - min_x)/(max_x-min_x)
        el[1] = (el[1] - min_y)/(max_y-min_y)
    return seq
def read_ip(path):
    seq = []
    with open(path, 'r') as file:
        contents = file.read()
        items = contents.split()
        for i in range(1,len(items),2):
            seq.append([float(items[i]),float(items[i+1])])
        # train.append(normalize(seq))
    return normalize(seq)

for ch in chars:
    for f in os.listdir("Group24/Handwriting_Data/" + str(ch)+"/train"):
        train.append(read_ip("Group24/Handwriting_Data/" + str(ch)+"/train/" + str(f)))
        z_train.append(str(ch))
    for f in os.listdir("Group24/Handwriting_Data/" + str(ch)+"/dev"):
        test.append(read_ip("Group24/Handwriting_Data/" + str(ch) + "/dev/" + str(f)))
        z_test.append(str(ch))

In [4]:
label_map = {'a':0, 'bA':1, 'dA':2, 'lA':3, 'tA':4}
# label_map = {"a":0, "ai":1, "bA":2, "lA":3, "tA":4}
z_train_onehot = tf.keras.utils.to_categorical([label_map[x] for x in z_train], 5)
z_test_onehot = tf.keras.utils.to_categorical([label_map[x] for x in z_test], 5)

Flattening

#### RNN Model

In [ ]:
my_tensor = tf.ragged.constant(train)
model = Sequential()
model.add(SimpleRNN(32, batch_input_shape=(None, None, 2), return_sequences=True))
model.add(SimpleRNN(32, activation="relu", return_sequences=True))
# model.add(SimpleRNN(64, activation="relu", return_sequences=True))
model.add(SimpleRNN(32, activation="relu", return_sequences=False))
model.add(Dense(32, activation="relu"))
model.add(Dense(5, activation="softmax"))
loss_fn = tf.keras.losses.CategoricalCrossentropy()

adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model.compile(loss="categorical_crossentropy", optimizer=adam_optimizer, metrics=['accuracy']) 
# Train the model
my_callbacks = [ EarlyStopping(monitor='loss', min_delta=1e-4, patience=10),    TensorBoard(log_dir=f'./logdir/Q1/test')]
model_fit = model.fit(my_tensor, z_train_onehot, batch_size=len(train), epochs=10000, verbose=1, validation_split=0.0,callbacks=my_callbacks, shuffle=False, validation_batch_size=None)

hist_metric = 'accuracy'
print(f'epochs: {len(model_fit.history[hist_metric])}, acc: {model_fit.history[hist_metric][-1]}\n')
model.save(f'models/Q1/test.tf')

test_tensor = tf.ragged.constant(test)
loss, accuracy = model.evaluate(test_tensor,z_test_onehot)

print("Test loss = ", loss)
print("Test accuracy = ", accuracy)

#### LSTM Model

In [5]:
train_tensor = tf.ragged.constant(train)
test_tensor = tf.ragged.constant(test)

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, batch_input_shape=(None,None,2)),
    tf.keras.layers.Dense(5, activation='softmax')
])

loss_fn = tf.keras.losses.CategoricalCrossentropy()

adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model.compile(loss="categorical_crossentropy", optimizer=adam_optimizer, metrics=['accuracy']) 

my_callbacks = [ EarlyStopping(monitor='loss', min_delta=1e-4, patience=10),    TensorBoard(log_dir=f'./logdir/Q1/LSTM')]
model_fit = model.fit(train_tensor, z_train_onehot, batch_size=len(train), epochs=10000, verbose=1, validation_split=0.0,callbacks=my_callbacks, shuffle=False, validation_batch_size=None)

hist_metric = 'accuracy'
print(f'epochs: {len(model_fit.history[hist_metric])}, acc: {model_fit.history[hist_metric][-1]}\n')
model.save(f'models/Q1/LSTM.tf')

loss, accuracy = model.evaluate(test_tensor,z_test_onehot)

print("Test loss = ", loss)
print("Test accuracy = ", accuracy)

2023-05-09 17:10:03.834212: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-09 17:10:03.834268: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-09 17:10:03.834311: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (piyush-HP-Spectre-x360-Convertible-13-aw2xxx): /proc/driver/nvidia/version does not exist
2023-05-09 17:10:03.978847: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10000
1/1 [==============================] - 1s 1s/step - loss: 1.6222 - accuracy: 0.2018
Epoch 2/10000
1/1 [==============================] - 0s 135ms/step - loss: 1.6133 - accuracy: 0.2018
Epoch 3/10000
1/1 [==============================] - 0s 137ms/step - loss: 1.6055 - accuracy: 0.2018
Epoch 4/10000
1/1 [==============================] - 0s 134ms/step - loss: 1.5988 - accuracy: 0.2018
Epoch 5/10000
1/1 [==============================] - 0s 135ms/step - loss: 1.5929 - accuracy: 0.2018
Epoch 6/10000
1/1 [==============================] - 0s 134ms/step - loss: 1.5877 - accuracy: 0.2018
Epoch 7/10000
1/1 [==============================] - 0s 136ms/step - loss: 1.5831 - accuracy: 0.2018
Epoch 8/10000
1/1 [==============================] - 0s 138ms/step - loss: 1.5790 - accuracy: 0.1930
Epoch 9/10000
1/1 [==============================] - 0s 157ms/step - loss: 1.5750 - accuracy: 0.2076
Epoch 10/10000
1/1 [==============================] - 0s 140ms/step - loss: 1.5711 - accuracy:

INFO:tensorflow:Assets written to: models/Q1/LSTM.tf/assets


INFO:tensorflow:Assets written to: models/Q1/LSTM.tf/assets


4/4 [==============================] - 0s 7ms/step - loss: 0.2606 - accuracy: 0.9300
Test loss =  0.26064276695251465
Test accuracy =  0.9300000071525574
